## Importação das bibliotecas

In [1]:
import numpy as np
import pandas as pd
import plotly
import plotly.offline as py
import yfinance as yf
import datetime
import plotly.graph_objs as go
import openpyxl as xl
import yagmail

## Sem esse comando os graficos não sao exibidos no notebook

In [2]:
from plotly.offline import plot, iplot
plotly.offline.init_notebook_mode(connected=True)

## Recebe os inputs do usuário e busca as cotações na base de dados do Yahoo

In [3]:
# Recebe o input do usuario com o nome da ação
stock = input('Digite o código da ação que deseja analisar')
stock = stock + '.SA'
# Recebe do usuário o valor das médias 
num_mm_short = int(input('Digite um valor númerico para a média móvel curta'))
num_mm_long = int(input('Digite um valor numerico para a média móvel longa'))
mean_short = 'MM' + str(num_mm_short)
mean_long = 'MM' + str(num_mm_long)
# Define o lote padrão de 100 ações
capital = int(input('Digite o capital disponível para essa estratégia'))

# Define o periodo de busca do historico de preços
ini_date = input('Digite a data inicial da busca.(Formato:DD-MM-AAAA')
end_date = input('Digite a data inicial da busca.(Formato:DD-MM-AAAA')
ini_date = ini_date.replace('/','-')
end_date = end_date.replace('/','-')
ini_date = datetime.datetime.strptime(ini_date, "%d-%m-%Y")
end_date = datetime.datetime.strptime(end_date, "%d-%m-%Y")

# Digitação do email e senha caso queira receber alerta de entrada por email
email = input('Digite seu email')
password = input('Digite a senha do seu email. Caso não queria receber alerta via e-mail digitar 0 para a senha')
yag = yagmail.SMTP(email,password)

df_stock_prices = yf.download(stock, start=ini_date, end=end_date, interval='1d')
df_stock_prices.reset_index(drop=False, inplace=True)

df_stock_prices.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,22.549999,24.200001,22.280001,24.059999,23.285252,104534800
1,2019-01-03,23.959999,24.820000,23.799999,24.650000,23.856251,95206400
2,2019-01-04,24.850000,24.940001,24.469999,24.719999,23.923996,72119800
3,2019-01-07,24.850000,25.920000,24.700001,25.110001,24.301439,121711900
4,2019-01-08,25.400000,25.420000,24.770000,24.959999,24.156267,68761800


## Cria as colunas de média móvel no Data Frame e preenche com zero os campos com 'NaN'

In [4]:
df_stock_prices['MM_SHORT'] = df_stock_prices.Close.rolling(window=num_mm_short).mean()
df_stock_prices['MM_LONG'] = df_stock_prices.Close.rolling(window=num_mm_long).mean()
df_stock_prices['SPREAD_MM'] = df_stock_prices['MM_SHORT'] - df_stock_prices['MM_LONG']

df_stock_prices['Start'] = 0.00
df_stock_prices['Stop'] = 0.00
df_stock_prices['Profit'] = 0.00

df_stock_prices['MM_SHORT'] = df_stock_prices['MM_SHORT'].fillna(0)
df_stock_prices['MM_LONG'] = df_stock_prices['MM_LONG'].fillna(0)
# Excluindo do Data frame os registros com a média móvel zerada pois não dá pra fazer backtest nesses registros
df_stock_prices = df_stock_prices.loc[df_stock_prices['MM_LONG']!=0]
df_stock_prices.head(25)

,Date,Open,High,Low,Close,Adj Close,Volume,MM_SHORT,MM_LONG,SPREAD_MM,Start,Stop,Profit
20,2019-01-31,25.870001,25.959999,25.559999,25.580000,24.756304,63253600,25.374445,25.107619,0.266826,0.0,0.0,0.0
21,2019-02-01,25.500000,25.850000,25.370001,25.799999,24.969221,47957400,25.418889,25.190476,0.228413,0.0,0.0,0.0
22,2019-02-04,25.840000,26.120001,25.600000,26.030001,25.191813,53874900,25.474445,25.256191,0.218254,0.0,0.0,0.0
23,2019-02-05,25.910000,26.100000,25.809999,26.080000,25.240204,40989100,25.580000,25.320952,0.259048,0.0,0.0,0.0
24,2019-02-06,25.799999,25.840000,25.320000,25.490000,24.669203,64527000,25.586667,25.339048,0.247619,0.0,0.0,0.0
25,2019-02-07,25.600000,25.850000,24.660000,25.090000,24.282085,55614100,25.536667,25.345238,0.191429,0.0,0.0,0.0
26,2019-02-08,24.799999,25.280001,24.730000,25.129999,24.320793,40153900,25.576667,25.328572,0.248095,0.0,0.0,0.0
27,2019-02-11,25.170000,25.170000,24.540001,24.840000,24.040133,39617300,25.517778,25.308571,0.209206,0.0,0.0,0.0
28,2019-02-12,25.320000,25.850000,25.230000,25.719999,24.891796,72467200,25.528889,25.343333,0.185555,0.0,0.0,0.0
29,2019-02-13,26.020000,26.209999,25.850000,26.049999,25.211170,69042200,25.581111,25.400476,0.180635,0.0,0.0,0.0


## Plota o gráfico com o preço de fechamento e as médias móveis

In [5]:
close_prices = go.Scatter(
                            x = df_stock_prices['Date'],
                            y = df_stock_prices['Close'],
                            name = stock + 'Preço de Fechamento',
                            line = dict(color = '#330000'),
                            opacity = 1.0)

close_prices2 = go.Candlestick(
                                x = df_stock_prices['Date'],
                                open = df_stock_prices['Open'],
                                high = df_stock_prices['High'],
                                low = df_stock_prices['Low'],
                                close = df_stock_prices['Close'])

mm_s = go.Scatter(
                    x = df_stock_prices['Date'],
                    y = df_stock_prices['MM_SHORT'],
                    name = 'Média Móvel Curta',
                    line = dict(color = '#B2FF66'),
                    opacity = 1.0)

mm_l = go.Scatter(
                    x = df_stock_prices['Date'],
                    y = df_stock_prices['MM_LONG'],
                    name = 'Média Móvel Longa',
                    line = dict(color = '#FF00FF'),
                    opacity = 1.0)

layout = go.Layout(
                    autosize=False,
                    legend_orientation='h',
                    width=1000,
                    height=400)

data_prices = [close_prices,mm_s,mm_l]

fig = go.Figure(data=data_prices, layout=layout)

py.iplot(fig)

## Faz o backtest da estratégia 

In [11]:
start_price = 0
last_spread = 0
lowest_price = 0
drawdown = 0
drawdown2 = 0
payoff = 0
date_today = datetime.date.today() - datetime.timedelta(days=3)
send_mail = False
fee = 5
fee_value = 0
income_tax = 0


for row in df_stock_prices.itertuples():
    lote = capital / row.Close
    verify_today = row.Date >= date_today
    #print(verify_today)
    #print(lote)
    if (row.MM_SHORT > row.MM_LONG) & (start_price < 0.01):
        #print(start_price)
        df_stock_prices.at[row.Index, 'Start'] = row.Close
        start_price = row.Close
        if verify_today == True:
            yag.send(email, 'Estratégia Cruzamento MM - Algo Python', 'Entrada pela estratégia de cruzmento de MM foi detectada')
        send_mail = False
    elif (row.MM_SHORT < row.MM_LONG) & (start_price < 0.01):
        #print(start_price)
        df_stock_prices.at[row.Index, 'Start'] = row.Close
        start_price = row.Close
    elif (row.MM_SHORT > row.MM_LONG) & (start_price > 0):
        #print(start_price)
        if last_spread < 0:
            df_stock_prices.at[row.Index, 'Profit'] = (start_price - row.Close) * lote
            df_stock_prices.at[row.Index, 'Stop'] = row.Close
            df_stock_prices.at[row.Index, 'Start'] = row.Close
            start_price = row.Start
        else:
            df_stock_prices.at[row.Index, 'Start'] = start_price
    elif (row.MM_SHORT < row.MM_LONG) & (start_price > 0):
        #print(start_price)
        if last_spread > 0:
            df_stock_prices.at[row.Index, 'Profit'] = (row.Close - start_price) * lote
            df_stock_prices.at[row.Index, 'Stop'] = row.Close
            if drawdown != 0:
                drawdown2 = (start_price - lowest_price) / start_price
                if drawdown2 > drawdown:
                    drawdown = drawdown2
                    drawdown2 = 0
            else:
                drawdown = (start_price - lowest_price) / start_price
            df_stock_prices.at[row.Index, 'Start'] = row.Close
            start_price = row.Start
            if verify_today == True:
                yag.send(email, 'Estratégia Cruzamento MM - Algo Python', 'Encerrada operação pela estratégia de cruzamento de MM.')
            #send_mail = False
        else:
            df_stock_prices.at[row.Index, 'Start'] = start_price
            if verify_today == True:
                yag.send(email, 'Estratégia Cruzamento MM - Algo Python', 'Entrada pela estratégia de cruzamento de MM foi detectada')
            #send_mail = False
    last_spread = row.SPREAD_MM

df_stock_prices.head(10)
print(drawdown)

1.0


## Print mostrando o resultado da estrategia

In [7]:
print('O resultado da estratégia é R$ %7.2f' % df_stock_prices['Profit'].sum())

O resultado da estratégia é R$ -168.02


## Exporta o data frame para uma planilha do excel

In [8]:
df_stock_prices.to_excel('cruzamento_mm.xlsx', sheet_name='Cross_MM', index=False)

In [9]:
# Armazena numero de entradas da estratégia
df_stock_prices_count = df_stock_prices.loc[df_stock_prices['Profit']!=0].count()
entry_count = df_stock_prices_count['Profit']

# Armazena número de entradas com lucro
df_stock_prices_count = df_stock_prices.loc[df_stock_prices['Profit']>0].count()
entry_count_profit = df_stock_prices_count['Profit']

# Armazena número de entradas com prejuizo
df_stock_prices_count = df_stock_prices.loc[df_stock_prices['Profit']<0].count()
entry_count_loss = df_stock_prices_count['Profit']

# Armazena resultado financeiro das entradas com lucro
df_stock_prices_sum = df_stock_prices.loc[df_stock_prices['Profit']>0].sum()
entry_sum_profit = df_stock_prices_sum['Profit']

# Armazena resultado financeiro das entradas com prejuizo
df_stock_prices_sum = df_stock_prices.loc[df_stock_prices['Profit']<0].sum()
entry_sum_loss = df_stock_prices_sum['Profit']

# Payoff
payoff = (entry_sum_profit / entry_count_profit) / ((entry_sum_loss / entry_count_loss) * -1)
# Taxa de acerto
hit_rate = entry_count_profit / entry_count
# Expectativa matemática
math_expec = payoff * (1 - hit_rate)
# Corretagem
fee_value = (fee * entry_count) * 2
# Se houver lucro calcula o valor do IR
if df_stock_prices['Profit'].sum() > 0:
    income_tax = df_stock_prices['Profit'].sum() * 0.2

print(hit_rate)
print(payoff)
print(math_expec)
print('O número de entradas executadas foi de %7.0f' % entry_count)


0.36363636363636365
1.5529165667328397
0.9882196333754435
O número de entradas executadas foi de      11


In [10]:
# Se o arquivo existe ele será excluido para ser criado novamente
if os.path.exists('cruzamento_mm.xlsx'):
  os.remove('cruzamento_mm.xlsx')
else:
  print("The file does not exist")

# Exporta o data frame para um arquivo no Excel
df_stock_prices.to_excel('cruzamento_mm.xlsx', sheet_name='cruzamento_mm', index=False)

# Cria um resumo do backtest em uma nova guia do Excel
wb = xl.load_workbook('cruzamento_mm.xlsx')

sheet = wb.create_sheet(index = 0 , title = "Resumo_Backtest") 

sheet.column_dimensions['A'].width = 30
sheet.column_dimensions['B'].width = 20
sheet['A1'] = 'Resumo do Backtest'
sheet['A2'] = 'Periodo avaliado:'
sheet['B2'] = str(ini_date) + ' até ' + str(end_date)
sheet['A3'] = 'Papel:'
sheet['B3'] = stock.upper()
sheet['A4'] = 'Capital:'
sheet['B4'] = capital
sheet['B4'].style = 'Comma'
sheet['A5'] = 'Média Móvel Curta:'
sheet['B5'] = num_mm_short
sheet['A6'] = 'Média Móvel Longa'
sheet['B6'] = num_mm_long
sheet['A7'] = 'Resultado Financeiro: R$'
sheet['B7'] = df_stock_prices['Profit'].sum()
sheet['B7'].style = 'Comma'
sheet['A8'] = 'Rentabilidade:'
sheet['B8'] = df_stock_prices['Profit'].sum() / capital
sheet['B8'].style = 'Percent'
sheet['A9'] = 'Número de entradas Executadas: '
sheet['B9'] = entry_count
sheet['A10'] = 'Número de operações Vencedoras:'
sheet['B10'] = entry_count_profit
sheet['A11'] = 'Número de operações Perdedoras:'
sheet['B11'] = entry_count_loss
sheet['A12'] = 'O payoff da estratégia é:'
sheet['B12'] = payoff
sheet['B12'].style = 'Comma'
sheet['A13'] = 'O drawdown máximo da estratégia é de:'
sheet['B13'] = drawdown
sheet['B13'].style = 'Percent'
sheet['A14'] = 'A taxa de acerto da estratégia é de:'
sheet['B14'] = hit_rate
sheet['B14'].style = 'Percent'
sheet['A15'] = 'A expectativa matemática da estratégia é de:'
sheet['B15'] = math_expec
sheet['B15'].style = 'Comma'
sheet['A16'] = 'Valor gasto com corretagem:'
sheet['B16'] = fee_value
sheet['B16'].style = 'Comma'
sheet['A17'] = 'O valor de IR é:'
sheet['B17'] = income_tax
sheet['B17'].style = 'Comma'
sheet['A18'] = 'Resultado líquido após corretagem e IR:'
sheet['B18'] = df_stock_prices['Profit'].sum() - fee_value - income_tax
sheet['B18'].style = 'Comma'
wb.save('cruzamento_mm.xlsx')